
## **1. Introduzione Tecnica: Il Motore sotto il Cofano**

Molti praticanti usano XGBoost come una "black box", ma l'efficacia del tuning dipende dalla comprensione di due pilastri: l'approccio matematico di **secondo ordine** e l'**ottimizzazione di sistema**.

### **1.1 Oltre il classico Gradient Boosting (GBM)**

Il Gradient Boosting standard è un metodo "additivo". Invece di ottimizzare i parametri di un singolo modello complesso (come in una rete neurale), costruiamo un insieme di modelli "deboli" (alberi decisionali) in sequenza.

Matematicamente, la predizione al passo $t$ per l'istanza $i$ è data da:

$$\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + \eta f_t(x_i)$$

Dove:
* $\hat{y}_i^{(t-1)}$ è la predizione accumulata fino al passo precedente.
* $f_t(x_i)$ è il nuovo albero che stiamo aggiungendo.
* $\eta$ è il learning rate.



**La differenza cruciale:**
In un GBM standard, il nuovo albero $f_t$ viene addestrato per predire i **residui** (o il gradiente negativo della loss) del modello precedente. Usa il metodo della *Discesa del Gradiente* (Primo Ordine).

**XGBoost**, invece, utilizza il **Metodo di Newton** (Secondo Ordine). Quando ottimizza la funzione obiettivo per trovare il miglior albero successivo, non guarda solo la pendenza (gradiente), ma anche la **curvatura** della funzione di loss.

### **1.2 L'Espansione di Taylor: Il cuore di XGBoost**

Per capire i parametri che vedremo dopo (come `min_child_weight`), dobbiamo guardare l'approssimazione della Loss Function che XGBoost utilizza.

XGBoost approssima la funzione obiettivo usando l'**Espansione di Taylor di secondo ordine**:

$$\mathcal{L}^{(t)} \approx \sum_{i=1}^n \left[ l(y_i, \hat{y}^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i) \right] + \Omega(f_t)$$

Qui risiedono i due concetti più importanti per un utente avanzato:

1.  **$g_i$ (Gradiente):** La derivata prima (la direzione verso cui muoversi).
2.  **$h_i$ (Hessiana):** La derivata seconda (la curvatura). Rappresenta quanto siamo "sicuri" o quanto rapidamente sta cambiando il gradiente.

> **Insight da Esperto:** Quando tunerai `min_child_weight`, starai letteralmente impostando una soglia sulla somma delle Hessiane ($h_i$) in una foglia. In una regressione MSE, l'Hessiana è costante ($2$), quindi `min_child_weight` è semplicemente il numero di campioni. Ma in una classificazione logistica, l'Hessiana diventa piccola quando il modello è molto sicuro (probabilità vicine a 0 o 1). Ecco perché XGBoost è così preciso: **pesa gli errori in base alla "sicurezza" del modello corrente.**

### **1.3 Regolarizzazione Integrata ($\Omega$)**

A differenza di altre librerie che applicano la regolarizzazione come ripensamento (post-pruning), XGBoost include il termine di regolarizzazione $\Omega(f_t)$ direttamente nella funzione obiettivo durante la costruzione dell'albero:

$$\Omega(f) = \gamma T + \frac{1}{2}\lambda ||w||^2$$

* **$\gamma T$**: Penalizza il numero di foglie ($T$). Più foglie = costo maggiore. Questo è controllato dal parametro `gamma`.
* **$\lambda ||w||^2$**: Penalizza la grandezza dei pesi delle foglie ($w$, i valori predetti nelle foglie). Questo è controllato dal parametro `lambda`.

L'albero non cresce se il guadagno di loss non supera la penalità della complessità.

### **1.4 Ottimizzazione di Sistema (Engineering)**

XGBoost non è famoso solo per la matematica, ma per come gestisce l'hardware:

* **Block Structure & Parallelization:** Un errore comune è pensare che XGBoost costruisca alberi in parallelo. Non può (è sequenziale). Invece, parallelizza la **costruzione dei nodi**. I dati sono pre-ordinati e salvati in blocchi di memoria (CSC format), permettendo a più thread di calcolare i migliori split simultaneamente.
* **Weighted Quantile Sketch:** Per dataset enormi, è impossibile testare ogni possibile valore di split. XGBoost usa un algoritmo di sketch approssimato per trovare i migliori punti di taglio candidati, pesati dall'Hessiana ($h_i$).
* **Sparsity Awareness:** XGBoost impara una "direzione di default" per ogni nodo. Se un dato è mancante (NaN), viene instradato automaticamente nella direzione che minimizza l'errore di training.


## **2. Anatomia Pratica degli Iperparametri (Il "Booster")**

In un contesto avanzato, non si settano i parametri a caso. Bisogna vedere gli iperparametri come manopole che controllano tre aspetti: **Capacità del modello**, **Robustezza al rumore** e **Velocità di convergenza**.

Ecco come un esperto configura il "Booster" (`gbtree`), andando oltre i valori di default.

### **2.1 Controllo della Struttura (Il freno e l'acceleratore)**

Questi parametri definiscono la "forma" fisica dei tuoi alberi.

* **`max_depth` (Profondità dell'albero)**
    * **Cosa fa davvero:** Controlla la complessità delle interazioni tra le feature che il modello può apprendere. Un albero di profondità $N$ può catturare interazioni fino a $N$ variabili.
    * **Utilizzo Avanzato:**
        * **Non esagerare:** In XGBoost, a differenza di Random Forest, gli alberi non devono essere profondi. Spesso un range **3-6** è ottimale.
        * **Segnale vs Rumore:** Se aumenti la profondità sopra a 8-10, stai quasi certamente memorizzando il rumore, a meno che tu non abbia milioni di righe e interazioni feature estremamente complesse.
        * **Interazione:** È fortemente correlato a `min_child_weight`. Se alzi la profondità, *devi* alzare `min_child_weight` per evitare foglie con pochi dati.

* **`min_child_weight` (Il "Filtro anti-rumore")**
    * **Cosa fa davvero:** È la somma minima dei pesi (hessiana) necessaria per mantenere un nodo figlio. In pratica (per regressione/classificazione standard), puoi pensarlo come il numero minimo di istanze necessarie in una foglia per validare uno split.
    * **Utilizzo Avanzato:**
        * **Dataset sbilanciati:** Questo è il parametro più critico. Se hai classi rare, un valore alto impedirà al modello di isolarle.
        * **Dataset rumorosi:** Se il tuo dataset ha molto rumore (es. dati finanziari o sensori IoT), alza questo valore (es. 10, 20 o anche 100). Costringe l'albero a fare split solo su pattern molto "solidi" e frequenti.
        * **Rule of Thumb:** Inizia con 1. Se vedi overfitting massiccio (Train score >> Test score), prova subito a saltare a 5 o 10.

* **`gamma` (o `min_split_loss`) - Il Pruning Aggressivo**
    * **Cosa fa davvero:** È una soglia "hard". Se lo split non riduce la loss function di almeno `gamma`, lo split non avviene. È una regolarizzazione che agisce *durante* la costruzione, non dopo.
    * **Utilizzo Avanzato:**
        * **Default vs Realtà:** Il default è 0 (crescita greedy).
        * **Quando usarlo:** È utilissimo quando il modello continua a creare rami inutili che migliorano di pochissimo la performance. Impostare un gamma basso (es. 0.1 - 0.5) rende il modello molto più conservativo (ottimo per evitare overfitting in produzione).
        * **Tuning:** È difficile da tunare con GridSearch perché dipende dalla scala della tua loss. Meglio lasciarlo a 0 all'inizio e alzarlo solo se la regolarizzazione standard (`lambda`/`alpha`) non basta.

### **2.2 Campionamento Stocastico (La diversità)**

Questi parametri introducono casualità. Senza di questi, XGBoost è deterministico. La casualità riduce la correlazione tra gli alberi, migliorando l'ensemble (meno varianza).

* **`subsample` (Righe)**
    * **Pratica:** Percentuale di righe campionate per costruire ogni albero.
    * **Sweet Spot:** Generalmente tra **0.6 e 0.9**.
    * **Warning:** Non scendere mai sotto 0.5 a meno che il dataset non sia enorme. Impostarlo a 1.0 (default) spesso porta a overfitting perché ogni albero vede esattamente gli stessi dati.

* **`colsample_bytree` (Colonne)**
    * **Pratica:** Percentuale di colonne (feature) scelte a caso per costruire ogni albero. Simile al `max_features` di Random Forest.
    * **Utilizzo Avanzato:**
        * Questo è spesso **più efficace della regolarizzazione L1/L2**.
        * Se hai molte feature collineari (ridondanti), abbassa questo valore (es. 0.6). Costringe gli alberi a usare feature diverse, rendendo il modello finale più robusto se una feature dovesse "rompersi" in produzione.



---

### **2.3 La Strategia di Tuning Professionale ("The Recipe")**

Non lanciare una GridSearch cieca su tutti i parametri. Sprechi CPU. Usa questo approccio a imbuto:

1.  **Fase 1: Baseline Veloce**
    * Fissa un `learning_rate` alto (es. 0.1 o 0.2) per addestrare velocemente.
    * Trova il numero ottimale di alberi (`n_estimators`) usando `early_stopping`.

2.  **Fase 2: Struttura dell'Albero (Macro-tuning)**
    * Tuna `max_depth` e `min_child_weight` insieme. Sono i parametri che impattano di più sul risultato.
    * *Esempio:* GridSearch su Depth [3, 5, 7, 9] e Child Weight [1, 3, 5].

3.  **Fase 3: Regolazione fine (Micro-tuning)**
    * Tuna `gamma` per potare i rami inutili.
    * Tuna `subsample` e `colsample_bytree` per aggiungere robustezza.

4.  **Fase 4: Il "Grand Finale" (Lower Rate, More Trees)**
    * Una volta trovati i parametri strutturali, abbassa il `learning_rate` (es. a 0.01 o 0.005).
    * Aumenta proporzionalmente `n_estimators`.
    * *Nota da esperto:* Questo passaggio da solo regala spesso un boost di 1-2% di performance, perché permette all'algoritmo di convergere verso il minimo globale con passi più fini, riducendo l'errore residuo.



## **3. Parametri di Apprendimento e Regolarizzazione**

Qui gestiamo due aspetti critici: **come** il modello impara dai propri errori (Learning Rate) e **come** evita di dare troppa importanza a feature rumorose (Regolarizzazione).

### **3.1 Il Motore del Boosting: `eta` e `n_estimators`**

Questi due parametri vivono in simbiosi. Non puoi modificarne uno senza considerare l'altro.

* **`eta` (o `learning_rate`)**
    * **Concetto Pratico:** È la "dimensione del passo". Dopo ogni albero, XGBoost non aggiunge l'intero valore della predizione, ma lo moltiplica per `eta`. Questo riduce l'impatto di ogni singolo albero, lasciando spazio agli alberi successivi per correggere gli errori.
    * **Perché è importante:** Un learning rate basso rende il modello più robusto all'overfitting, poiché la costruzione del modello finale è più graduale e meno dipendente dai primi alberi (che potrebbero aver memorizzato rumore).

* **`n_estimators` (o `num_boost_round`)**
    * **Concetto Pratico:** Il numero totale di alberi sequenziali da costruire.



#### **La Strategia Professionale: "Shrinkage"**
La regola d'oro nell'industria non è cercare il "learning rate magico", ma seguire questa procedura:

1.  **Fase di Tuning:** Usa un `eta` alto (es. **0.1** o **0.2**) e un numero di stimatori basso/medio. Questo ti permette di fare decine di test di `max_depth` e `subsample` in pochi minuti invece che ore.
2.  **Fase di Produzione:** Una volta trovata la struttura ideale degli alberi, applica la tecnica dello **Shrinkage**:
    * Riduci `eta` di un fattore $X$ (es. da 0.1 a **0.01**).
    * Aumenta `n_estimators` dello stesso fattore $X$.
    * *Risultato:* Il modello impiegherà più tempo ad addestrarsi, ma l'errore di generalizzazione scenderà quasi sempre, migliorando la precisione finale.

> **Nota:** Usa sempre **Early Stopping** quando aumenti gli stimatori. Se imposti 10.000 alberi ma il modello smette di migliorare al 3.500esimo, l'early stopping fermerà il training, risparmiando ore di calcolo e prevenendo l'overfitting tardivo.

---

### **3.2 Regolarizzazione Esplicita (L1 & L2)**

Mentre `max_depth` limita la struttura, `lambda` e `alpha` limitano i **pesi** numerici assegnati alle foglie. Questo è fondamentale quando si hanno feature con alta varianza o dataset con molto rumore.

* **`lambda` (Regolarizzazione L2 - Ridge)**
    * **Default:** 1 (Attivo di default, a differenza di sklearn che spesso non regolarizza).
    * **Effetto Pratico:** "Schiaccia" i valori delle foglie verso zero in modo fluido. Penalizza i valori estremi.
    * **Quando usarlo:** È il tuo "scudo" standard. Se vedi che il modello dà pesi enormi a certe predizioni su pochi casi, aumenta `lambda`. Aiuta a gestire la multicollinearità (feature correlate).

* **`alpha` (Regolarizzazione L1 - Lasso)**
    * **Default:** 0.
    * **Effetto Pratico:** Forza i pesi delle feature inutili a diventare **esattamente zero**.
    * **Quando usarlo (Feature Selection Implicita):**
        * Se hai un dataset con **migliaia di feature** (es. One-Hot Encoding di variabili categoriche ad alta cardinalità o dati genomici) e sospetti che solo poche siano importanti.
        * Impostare `alpha` alto rende il modello "sparso" (più leggero e veloce in inferenza perché usa meno feature).

#### **Scenario d'uso: L1 vs L2**
* *Problema classico (es. Churn Prediction):* Usa **L2 (`lambda`)**. Vogliamo considerare tutte le variabili un po'.
* *Problema ad alta dimensione (es. Analisi del testo con TF-IDF):* Usa **L1 (`alpha`)**. Vogliamo eliminare le migliaia di parole che non servono a nulla.

---

**Sintesi del Punto 3:**
* **Learning Rate:** Tienilo basso in produzione (0.01 - 0.05).
* **Estimators:** Alzali quando abbassi il learning rate (e usa Early Stopping).
* **Alpha/Lambda:** Usa Alpha se vuoi selezionare feature, Lambda per stabilità generale.




## **4. Gestione di Scenari Complessi**

### **4.1 Dataset Sbilanciati (Imbalanced Learning)**
Quando la classe positiva (quella che ci interessa, es. "Frode" o "Guasto") è molto rara, il modello tende a ignorarla per massimizzare l'accuratezza globale (dicendo sempre "Non Frode").

Ecco come forzare XGBoost a prestare attenzione ai casi rari:

* **`scale_pos_weight` (Il Bilanciatore)**
    * **La Logica:** Modifica il calcolo del gradiente. Se questo parametro è > 1, gli errori commessi sulla classe positiva pesano di più durante l'aggiornamento dei pesi.
    * **Formula Magica:**
        $$scale\_pos\_weight = \frac{\text{Numero di Negativi}}{\text{Numero di Positivi}}$$
    * **Utilizzo Professionale:**
        * Calcola questo rapporto e inseriscilo nel modello. Spesso è più efficace e veloce delle tecniche di campionamento esterne (come SMOTE o Oversampling) perché non altera la distribuzione dei dati, ma solo la penalità matematica.
        * **Attenzione alle Probabilità:** Quando usi `scale_pos_weight`, le probabilità predette (`predict_proba`) non saranno più calibrate (saranno spostate verso l'alto). Se hai bisogno della probabilità reale (es. per calcolare il rischio finanziario esatto), dovrai ricalibrarle a valle.

* **`max_delta_step` (La cintura di sicurezza per la convergenza)**
    * **Il Problema:** In scenari estremamente sbilanciati, l'aggiornamento dei pesi di un singolo albero può essere enorme ("esplosivo"), portando a instabilità numerica.
    * **Soluzione:** `max_delta_step` pone un tetto massimo al cambiamento di peso (delta) di una singola foglia.
    * **Configurazione:** Il default è 0 (nessun tetto). Se hai problemi di convergenza con classi molto rare, impostalo tra **1 e 10**. Questo rende l'aggiornamento più conservativo e stabile.



### **4.2 Gestione dei Valori Mancanti (Sparsity Awareness)**

Molti ingegneri perdono tempo a imputare i valori mancanti (Mean, Median, KNN Imputation) prima di passare i dati a XGBoost. Spesso, **questo è un errore**.

* **Sparsity Aware Split Finding**
    * XGBoost gestisce i `NaN` (Not a Number) nativamente. Non li ignora, li *usa*.
    * **Come funziona:** Per ogni nodo (decisione) nell'albero, l'algoritmo testa due scenari:
        1.  Manda tutti i dati con valore mancante a **Sinistra**. Calcola il guadagno di info.
        2.  Manda tutti i dati con valore mancante a **Destra**. Calcola il guadagno di info.
    * La direzione che minimizza la loss viene "imparata" e salvata come **Default Direction**.
    * **Perché è geniale:** Spesso un dato mancante non è casuale (Missing Not At Random).
        * *Esempio:* In un dataset bancario, se il campo "Debito Pregresso" è vuoto (`NaN`), potrebbe significare che il cliente non ha mai avuto debiti (ottimo pagatore). Se lo riempi con la media, distruggi questa informazione. XGBoost invece impara che `NaN` -> "Ramo dei buoni pagatori".



#### **Best Practice sui Dati Mancanti:**
1.  **Non imputare nulla** inizialmente. Passa i `NaN` (o `np.nan` in Python) direttamente a XGBoost.
2.  Lascia che l'algoritmo scopra se l'assenza di informazione è essa stessa un'informazione (informative missingness).
3.  Imputa manualmente solo se sai per certo che il dato manca per un errore tecnico casuale e la media/mediana è una stima affidabile.




## **5. Tecniche Avanzate: Ingegnerizzare la Conoscenza**

### **5.1 Monotonic Constraints (Vincoli Monotoni)**
Spesso sappiamo a priori come una feature dovrebbe influenzare il target.
* *Esempio Immobiliare:* A parità di altre condizioni, se i metri quadri aumentano, il prezzo *deve* salire (o rimanere uguale). Non può scendere.
* *Il Problema:* Se il training set ha dei dati rumorosi (es. una casa grande svenduta per urgenza), un albero normale potrebbe imparare un "dip" (calo) di prezzo per metrature alte. Questo è overfitting su rumore locale.

**La Soluzione:**
Puoi forzare XGBoost a rispettare una relazione sempre crescente o decrescente per specifiche feature.

* **Parametro:** `monotone_constraints`
* **Valori:**
    * `1`: Relazione crescente (All'aumentare di X, Y aumenta o resta uguale).
    * `-1`: Relazione decrescente (All'aumentare di X, Y diminuisce o resta uguale).
    * `0`: Nessun vincolo.
* **Vantaggi:**
    1.  **Migliore Generalizzazione:** Il modello ignora il rumore che contraddice la logica nota.
    2.  **Explainability & Trust:** Quando spieghi il modello agli stakeholder, non vedranno comportamenti illogici (es. "Perché se guadagno di più la banca mi dà meno credito?").



### **5.2 Interaction Constraints (Vincoli di Interazione)**
Di default, XGBoost può combinare qualsiasi feature con qualsiasi altra. Ma in certi settori (es. Assicurativo, Credit Risk), alcune interazioni potrebbero essere vietate per legge (discriminazione) o prive di senso logico.

* **Parametro:** `interaction_constraints`
* **Come funziona:** Passi una lista di liste. Le feature presenti nella stessa sottolista possono interagire tra loro, ma non con feature di altre liste.
    * *Esempio:* `[[Feature_A, Feature_B], [Feature_C, Feature_D, Feature_E]]`.
    * Qui l'albero può fare split su A e poi su B nello stesso ramo. Ma se ha fatto split su C, non può scendere e fare uno split su A.
* **Uso Avanzato:** Riduce drasticamente lo spazio delle ipotesi, prevenendo il modello dal trovare correlazioni spurie complesse che non esistono nella realtà.

### **5.3 Custom Objective Functions (Loss Personalizzate)**
Questo è il livello "Gran Maestro". A volte `RMSE` (Regressione) o `LogLoss` (Classificazione) non riflettono il vero obiettivo di business.

* **Scenario (Asymmetric Loss):** Immagina di predire la domanda di magazzino.
    * Sottostimare la domanda (Stock-out) costa 1000€ in vendite perse.
    * Sovrastimare la domanda (Over-stock) costa 50€ di stoccaggio.
    * L'errore quadratico medio (MSE) tratterebbe +10 e -10 allo stesso modo. Per il business, -10 è un disastro.

* **Implementazione:**
    XGBoost permette di definire una funzione Python personalizzata che calcola e restituisce due vettori per ogni istanza di training:
    1.  **Gradiente (Gradient):** La derivata prima (Direzione dell'errore).
    2.  **Hessiana (Hessian):** La derivata seconda (Curvatura/Peso dell'errore).

Definendo una funzione che penalizza fortemente il gradiente quando l'errore è negativo (stock-out) e poco quando è positivo, guidi l'apprendimento verso una strategia di "prudente sovrastima".

### **5.4 Supporto Nativo per Feature Categoriche**
Fino a poco tempo fa, dovevi fare *One-Hot Encoding* (OHE) prima di usare XGBoost. Questo creava matrici sparse enormi e alberi sbilanciati (perché per isolare la categoria "Z" servivano tanti split se usavi OHE).

* **La Novità:** XGBoost ora supporta `enable_categorical=True` (con tree method `hist` o `gpu_hist`).
* **Come funziona (Optimal Partitioning):** Invece di trattare le categorie come numeri, l'algoritmo cerca la partizione ottimale delle categorie in due gruppi a ogni nodo.
    * *Split:* "È la categoria {A, C, F}?" vs "È la categoria {B, D, E}?".
* **Vantaggio:**
    * Addestramento molto più veloce su dataset con alta cardinalità.
    * Spesso performance superiori rispetto a OHE o Label Encoding, specialmente con alberi poco profondi.





## **6. Strategie di Tuning e Best Practices (Il Metodo Optuna)**

### **6.1 Perché Optuna cambia le regole del gioco**

`GridSearch` prova tutte le combinazioni ciecamente. `Optuna` impara dal passato.
Usa un algoritmo chiamato **TPE (Tree-structured Parzen Estimator)**.

In parole semplici:

1.  Optuna lancia un training con parametri a caso.
2.  Osserva il risultato.
3.  Costruisce un modello probabilistico interno che dice: "Quando `max_depth` è basso e `eta` è alto, il modello fa schifo. Non proverò più lì. Invece, sembra che `subsample` alto funzioni bene, esplorerò di più quella zona."
4.  **Pruning (Potatura):** Se un trial (tentativo) sta andando male dopo 10 iterazioni, Optuna lo uccide subito. Non spreca risorse per arrivare alla fine di un training fallimentare.

### **6.2 Definire lo Spazio di Ricerca (Search Space)**

Prima del codice, ecco come un esperto definisce i range. Nota l'uso della scala logaritmica.

  * `learning_rate`: **Logaritmicamente** tra 1e-3 e 0.3. È fondamentale perché l'impatto varia per ordini di grandezza.
  * `max_depth`: Intero tra 3 e 10 (o 12).
  * `min_child_weight`: Intero tra 1 e 10 (per il rumore).
  * `subsample` / `colsample_bytree`: Float tra 0.5 e 1.0.
  * `reg_lambda` / `reg_alpha`: **Logaritmicamente** tra 1e-8 e 10.0.

### **6.3 Il Codice: Template Professionale per XGBoost + Optuna**

Questo script non è solo un esempio, è un template pronto per la produzione. Include il **Pruning**, che è la parte che accelera il tuning del 50-70%.

```python
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split

# 1. Definizione della Funzione Obiettivo
def objective(trial, X, y):
    
    # A. Split veloce per validazione interna al trial
    # Nota: In produzione, potresti usare StratifiedKFold qui dentro per maggiore robustezza
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25)
    
    # B. Definizione dello Spazio degli Iperparametri (Dynamic Search Space)
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic', # o 'reg:squarederror' per regressione
        'tree_method': 'hist',          # Usa 'gpu_hist' se hai una GPU! Velocizza di 10x.
        
        # Struttura dell'albero
        # Suggeriamo un intero per la profondità
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        # Suggeriamo un intero per il peso minimo (controllo rumore/outlier)
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        # Gamma: soglia minima di riduzione loss per split
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        
        # Campionamento (Stochastic Gradient Boosting)
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        
        # Regolarizzazione (Scala Logaritmica è cruciale qui)
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        
        # Learning Rate e Estimators
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': 1000 # Impostiamo un tetto alto, l'early stopping lo fermerà prima
    }

    # C. Inizializzazione del Pruner (Interruzione anticipata dei trial scarsi)
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-logloss")
    
    # D. Addestramento del Modello
    model = xgb.XGBClassifier(**param)
    
    model.fit(
        train_x, 
        train_y, 
        eval_set=[(valid_x, valid_y)], 
        eval_metric="logloss",
        early_stopping_rounds=50, # Ferma se non migliora per 50 round
        callbacks=[pruning_callback], # Collega Optuna a XGBoost
        verbose=False
    )
    
    # E. Predizione e calcolo metrica da ottimizzare
    preds = model.predict_proba(valid_x)[:, 1] # Probabilità classe 1
    loss = log_loss(valid_y, preds)
    
    return loss

# ---------------------------------------------------------
# Esecuzione dello Studio
# ---------------------------------------------------------

# Supponiamo di avere X e y caricati
# X, y = load_data(...) 

# 1. Creiamo lo studio (Direction: Minimize LogLoss)
study = optuna.create_study(
    direction='minimize', 
    sampler=optuna.samplers.TPESampler(), # Sampler Bayesiano standard
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5) # Uccide i trial peggiori della media
)

# 2. Avviamo l'ottimizzazione
# n_trials: Quante combinazioni provare
# timeout: Tempo massimo in secondi (es. 1 ora = 3600)
print("Inizio Tuning con Optuna...")
study.optimize(lambda trial: objective(trial, X, y), n_trials=100, timeout=3600)

# 3. Risultati
print("Migliori parametri:", study.best_params)
print("Miglior Loss:", study.best_value)

# 4. Importanza degli Iperparametri
# Optuna ti dice quali parametri hanno influito di più sul risultato!
optuna.visualization.plot_param_importances(study).show()
```

### **6.4 Analisi del Codice: I Dettagli che contano**

1.  **`suggest_float(..., log=True)`**: Questa è la chiave. Per parametri come `alpha` o `learning_rate`, la differenza tra 0.001 e 0.01 è enorme (10x), mentre tra 0.8 e 0.81 è nulla. La scala logaritmica permette a Optuna di esplorare gli ordini di grandezza in modo efficiente.
2.  **`XGBoostPruningCallback`**: Senza questo, Optuna aspetterebbe la fine dei 1000 alberi per ogni trial. Con questo, se al 50° albero la loss è peggiore della media degli altri trial, Optuna lancia un'eccezione, ferma il training e passa al prossimo set di parametri.
3.  **`objective` Function**: Nota come tutto (definizione parametri, training, valutazione) avviene dentro questa funzione. Optuna la chiama ripetutamente.
4.  **`MedianPruner`**: Una strategia di pruning semplice ma efficace. Se il trial corrente sta andando peggio della mediana dei trial precedenti allo stesso step, viene tagliato.

### **6.5 Workflow Finale: Dal Tuning alla Produzione**

Una volta che `study.best_params` ti restituisce il dizionario vincente, non hai finito.

1.  **Prendi i migliori parametri.**
2.  **Applica la strategia "Low Rate" (Shrinkage):**
      * Prendi il `learning_rate` suggerito da Optuna e dividilo (es. per 2 o per 5).
      * Aumenta `n_estimators` per compensare.
3.  **Rialilena sul dataset completo:** Ora usa tutto il dataset (Train + Validation) per addestrare il modello finale che andrà in produzione, usando il numero di step ottimale trovato.


In [7]:
import optuna
import xgboost as xgb
import numpy as np
import warnings
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Silenziamo alcuni warning di Optuna per pulizia
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ==========================================
# 1. Preparazione Dati
# ==========================================
print("--- Caricamento Dati ---")
data = load_breast_cancer()
X, y = data.data, data.target

# Split 1: Togliamo il 20% dei dati per il TEST finale (Dati mai visti né da XGBoost né da Optuna)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Calcolo scale_pos_weight per bilanciamento (best practice)
# Anche se questo dataset è abbastanza bilanciato, lo calcoliamo per rigore professionale
num_neg = np.sum(y_train_full == 0)
num_pos = np.sum(y_train_full == 1)
scale_pos_weight = num_neg / num_pos

print(f"Dataset Shape: {X.shape}")
print(f"Scale Pos Weight calcolato: {scale_pos_weight:.2f}")

# ==========================================
# 2. Baseline (Senza Tuning)
# ==========================================
print("\n--- Training Modello Baseline (Default) ---")
dtrain_full = xgb.DMatrix(X_train_full, label=y_train_full)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parametri standard
params_base = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'tree_method': 'hist', # Più veloce
    'random_state': 42
}

model_base = xgb.train(params_base, dtrain_full, num_boost_round=100)
preds_base = model_base.predict(dtest)
auc_base = roc_auc_score(y_test, preds_base)
print(f"Baseline AUC Score: {auc_base:.4f}")


# ==========================================
# 3. Ottimizzazione con Optuna
# ==========================================
print("\n--- Inizio Tuning con Optuna ---")

def objective(trial):
    # A. Split interno per Optuna (Train vs Validation)
    # Optuna usa questo valid set per decidere se i parametri sono buoni
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)

    # B. Definizione Spazio Iperparametri
    param = {
        'verbosity': 0,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss', # Metrica monitorata per il pruning
        'tree_method': 'hist',
        'scale_pos_weight': scale_pos_weight,
        
        # --- Parametri da Ottimizzare ---
        # 1. Struttura
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        
        # 2. Campionamento
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        
        # 3. Regolarizzazione
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        
        # 4. Learning
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
    }

    # Callback per il Pruning (Interrompe i trial scarsi)
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-logloss")

    # Training del trial
    model = xgb.train(
        param,
        dtrain,
        num_boost_round=1000, # Alto numero teorico
        evals=[(dvalid, "validation")],
        early_stopping_rounds=50, # Stop se non migliora
        callbacks=[pruning_callback],
        verbose_eval=False
    )

    # Predizione sul validation set interno
    preds = model.predict(dvalid)
    # Ottimizziamo la LogLoss (più bassa è meglio)
    # Nota: Optuna minimizza per default, log_loss è perfetta
    loss = classification_report(y_valid, preds > 0.5, output_dict=True)['accuracy'] # Trucco: ottimizziamo accuracy invertita o logloss diretta
    
    # Per semplicità in questo esempio usiamo logloss diretta
    # Importante: model.predict restituisce probabilità
    from sklearn.metrics import log_loss
    return log_loss(y_valid, preds)

# Creazione Studio
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, timeout=600) # 50 tentativi o 10 minuti

print("Migliori parametri trovati:")
print(study.best_params)


# ==========================================
# 4. Training Modello Finale (Best Params)
# ==========================================
print("\n--- Training Modello Finale Ottimizzato ---")

# Recuperiamo i migliori parametri
best_params = study.best_params

# Aggiungiamo i parametri fissi necessari
best_params['objective'] = 'binary:logistic'
best_params['eval_metric'] = 'logloss'
best_params['tree_method'] = 'hist'
best_params['scale_pos_weight'] = scale_pos_weight

# TECNICA DEL "LOW LEARNING RATE" (Shrinkage)
# Riduciamo il learning rate trovato e aumentiamo gli alberi per precisione massima
best_params['learning_rate'] = best_params['learning_rate'] / 2 
num_boost_round_final = 2000 

# Addestriamo su TUTTO il set di training (Train + Valid interno di Optuna)
model_opt = xgb.train(
    best_params, 
    dtrain_full, 
    num_boost_round=num_boost_round_final,
    evals=[(dtest, "test")], # Usiamo il test set solo per early stopping finale
    early_stopping_rounds=50,
    verbose_eval=False
)

# ==========================================
# 5. Confronto Finale
# ==========================================
print("\n--- RISULTATI FINALI SUL TEST SET ---")

# Predizioni
preds_opt = model_opt.predict(dtest)

# Metriche
auc_opt = roc_auc_score(y_test, preds_opt)
acc_base = accuracy_score(y_test, preds_base > 0.5)
acc_opt = accuracy_score(y_test, preds_opt > 0.5)

print(f"1. AUC Score Baseline:    {auc_base:.5f}")
print(f"2. AUC Score Ottimizzato: {auc_opt:.5f}")
print("-" * 30)
print(f"Miglioramento AUC:        {auc_opt - auc_base:+.5f}")
print("-" * 30)

# Visualizzazione importanza iperparametri (se in notebook)
# optuna.visualization.plot_param_importances(study).show()

--- Caricamento Dati ---
Dataset Shape: (569, 30)
Scale Pos Weight calcolato: 0.60

--- Training Modello Baseline (Default) ---
Baseline AUC Score: 0.9901

--- Inizio Tuning con Optuna ---
Migliori parametri trovati:
{'max_depth': 7, 'min_child_weight': 3, 'gamma': 0.9329375475579135, 'subsample': 0.6906717073754871, 'colsample_bytree': 0.7307749963219472, 'lambda': 0.1697008050569727, 'alpha': 0.00045746225925563064, 'learning_rate': 0.21288125193848467}

--- Training Modello Finale Ottimizzato ---

--- RISULTATI FINALI SUL TEST SET ---
1. AUC Score Baseline:    0.99008
2. AUC Score Ottimizzato: 0.99405
------------------------------
Miglioramento AUC:        +0.00397
------------------------------


In [8]:
import optuna
import xgboost as xgb
import numpy as np
import warnings
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Pulizia output
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ===========================
# 1️⃣ Caricamento dati
# ===========================
data = fetch_california_housing(as_frame=True)
df = data.frame
X = df.drop(columns=['MedHouseVal']).values
y = df['MedHouseVal'].values

# Log-transform target per stabilità numerica
y = np.log1p(y)

# Standard scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split finale
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dataset shape: {X.shape}, Target medio: {np.mean(np.expm1(y)):.2f}")

# ===========================
# 2️⃣ Baseline CPU (per confronto)
# ===========================
dtrain_full = xgb.DMatrix(X_train_full, label=y_train_full)
dtest = xgb.DMatrix(X_test, label=y_test)

params_base = {
    'objective': 'reg:squarederror',
    'tree_method': 'hist',
    'random_state': 42,
    'max_depth': 8,
    'eta': 0.1
}

model_base = xgb.train(params_base, dtrain_full, num_boost_round=200)
preds_base = np.expm1(model_base.predict(dtest))
rmse_base = np.sqrt(mean_squared_error(np.expm1(y_test), preds_base))
print(f"Baseline RMSE: {rmse_base:.4f}")

# ===========================
# 3️⃣ Funzione Objective per Optuna (GPU)
# ===========================
def objective(trial):
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    rmses = []

    for train_idx, valid_idx in kf.split(X_train_full):
        X_train, X_valid = X_train_full[train_idx], X_train_full[valid_idx]
        y_train, y_valid = y_train_full[train_idx], y_train_full[valid_idx]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)

        param = {
            'verbosity': 0,
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',

            # GPU
            'tree_method': 'gpu_hist',
            'predictor': 'gpu_predictor',

            # Alberi
            'max_depth': trial.suggest_int('max_depth', 5, 15),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 15),
            'gamma': trial.suggest_float('gamma', 1e-8, 5.0, log=True),

            # Randomness
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),

            # Regolarizzazione
            'lambda': trial.suggest_float('lambda', 1e-8, 50.0, log=True),
            'alpha': trial.suggest_float('alpha', 1e-8, 50.0, log=True),

            # Learning
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            'max_bin': trial.suggest_int('max_bin', 128, 512),

            'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
            'booster': 'gbtree'
        }

        pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")

        model = xgb.train(
            param,
            dtrain,
            num_boost_round=1000,
            evals=[(dvalid, "validation")],
            early_stopping_rounds=50,
            callbacks=[pruning_callback],
            verbose_eval=False
        )

        preds = model.predict(dvalid)
        rmse = np.sqrt(mean_squared_error(np.expm1(y_valid), np.expm1(preds)))
        rmses.append(rmse)

    return np.mean(rmses)

# ===========================
# 4️⃣ Run Optuna
# ===========================
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, timeout=600)  # 50 trial max 10 minuti

print("\nBest validation RMSE:", study.best_value)
print("Best parameters:", study.best_params)

# ===========================
# 5️⃣ Training modello finale GPU
# ===========================
best_params = study.best_params.copy()
best_params.update({
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
})

# Riduzione learning rate finale per "shrinkage"
best_params['learning_rate'] *= 0.5
num_round_final = 8000

dtrain_f = xgb.DMatrix(X_train_full, label=y_train_full)
dtest_f = xgb.DMatrix(X_test, label=y_test)

model_opt = xgb.train(
    best_params,
    dtrain_f,
    num_boost_round=num_round_final,
    evals=[(dtest_f, "validation")],
    early_stopping_rounds=200,
    verbose_eval=False
)

# ===========================
# 6️⃣ Valutazione finale
# ===========================
preds_opt = np.expm1(model_opt.predict(dtest_f))
rmse_opt = np.sqrt(mean_squared_error(np.expm1(y_test), preds_opt))

print("\n--- Confronto finale ---")
print(f"RMSE Baseline:    {rmse_base:.4f}")
print(f"RMSE Ottimizzato: {rmse_opt:.4f}")
print(f"--> Miglioramento Errore: {rmse_base - rmse_opt:.4f}")


Dataset shape: (20640, 8), Target medio: 2.07
Baseline RMSE: 0.4581

Best validation RMSE: 0.4612579778244934
Best parameters: {'max_depth': 7, 'min_child_weight': 5, 'gamma': 1.7700023226065948e-05, 'subsample': 0.673502300873134, 'colsample_bytree': 0.7857990792075301, 'lambda': 4.5475339914832475e-07, 'alpha': 3.922347665504824e-08, 'learning_rate': 0.0812110716460948, 'max_bin': 418, 'grow_policy': 'depthwise'}

--- Confronto finale ---
RMSE Baseline:    0.4581
RMSE Ottimizzato: 0.4380
--> Miglioramento Errore: 0.0201


--- Caricamento California Housing Dataset ---
Dataset Shape: (20640, 8)
Target medio: 2.07 (centinaia di k$)

--- Training Modello Baseline (Default) ---
Baseline RMSE: 0.4718

--- Inizio Tuning con Optuna ---

Miglior RMSE trovato da Optuna (Validation): 0.4667
Migliori parametri: {'max_depth': 6, 'min_child_weight': 1, 'gamma': 2.647729432344551e-06, 'subsample': 0.7057560887773185, 'colsample_bytree': 0.8261113044843484, 'lambda': 0.1073402821501809, 'alpha': 0.12334515017305431, 'learning_rate': 0.037390927995818786}

--- Training Modello Finale Ottimizzato ---

--- CONFRONTO FINALE ---
RMSE Baseline:    0.4718
RMSE Ottimizzato: 0.4362
--> Miglioramento Errore: 0.0356 (Minore è meglio)
------------------------------
R2 Score Baseline:    0.8301
R2 Score Ottimizzato: 0.8548
--> Varianza Spiegata Extra: +2.47%

### **Pro-Tip: Monotonic Constraints su questo Dataset**

Se volessi rendere questo modello "bullet-proof" per un'agenzia immobiliare reale, dovresti aggiungere un vincolo al parametro `AveRooms` (Numero medio di stanze).

Nella realtà, *a parità di altre condizioni* (stessa zona, stessa età), una casa con più stanze vale di più. Se il modello imparasse il contrario (magari per rumore statistico), sarebbe un errore grave.

Potresti aggiungere questo nel `param` finale:

```python
# Supponendo che 'AveRooms' sia la colonna indice 2
# 1 = crescente, 0 = nessun vincolo
# Imponiamo che più stanze = prezzo più alto (o uguale)
constraints = (0, 0, 1, 0, 0, 0, 0, 0) 
best_params['monotone_constraints'] = constraints
```

Questo peggiorerebbe leggermente l'RMSE matematico, ma renderebbe il modello infinitamente più affidabile nel mondo reale.